In [14]:
import numpy as np
from math import erf, exp, log, sqrt


def N(x: float) -> float:
    """
    This function computes the cumulative distribution function (CDF) for the standard normal distribution, N(x).
    :param x: The value to evaluate the CDF at
    :return: The cumulative probability up to x
    """
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def bs_price_call(S0: float, K: float, r:float, sigma: float, T: float) -> float:
    """
    This calculates the analytical price of a European call option using the Black-Scholes formula.

    :param S0: Current asset price
    :param K: Option strike price
    :param r: Risk-free interest rate
    :param sigma: Volatility of the asset
    :param T: Time to maturity in years
    :return: The price of the European call option
    """

    # Fix NPE where time to maturity or volatility are zero.
    if T <= 0 or sigma <= 0:
        return max(S0 - K, 0.0) * exp(-r * T)

    # Standard Black-Scholes formula parameters
    vol = sigma * sqrt(T)
    d1 = (log(S0 / K) + (r + 0.5 * sigma**2) * T) / vol
    d2 = d1 - vol

    # Return the analytical price
    return S0 * N(d1) - K * exp(-r * T) * N(d2)

def payoff_call(S_t, K):
    """
    Calculates the final payoff of a European call option

    :param S_t: The simulated asset prices at maturity
    :param K: The strike price
    :return: Array of payoff for each path being simulated
    """

    return np.maximum(S_t - K, 0.0)


def rk_step(s, dW, dt, r, sigma):
    """
    Performs a single rk time step.

    :param s: Array of current asset prices
    :param dW: Array of Wiener increments for the step
    :param dt: The time step size
    :param r: Risk-free rate
    :param sigma: Volatility
    :return: Asset price at the next time step
    """
     # Predictor step (Euler-Maruyama guess)
    Y = s + r * s *dt + sigma * s *dW + 0.5 * (sigma**2) *s* (dW**2 - dt)
    # Corrector step (using the predictor Y to refine the solution)
    return (s + 0.5 * (r * s + r * Y) * dt          # Trapezoidal approximation of the drift
            + sigma * s * dW                      # Diffusion term
            + 0.5 * (sigma**2) * s * (dW**2 - dt))  # rk correction term



def rk_step_to_maturity(S0: float, dW: np.ndarray, dt: float, r: float, sigma: float):
    """
    Simulates multiple asset price paths from t=0 to maturity (T) by using the function above.

    :param S0: Initial asset price
    :param dW: A 2D array of Wiener increments for all paths and steps.
    :param dt: The time step size
    :param r: Risk-free rate
    :param sigma: Volatility
    :return: The final asset price for each path at maturity
    """
    # Initializes all paths to the starting price S0
    s = np.full(dW.shape[0], S0, dtype=float)

    # Loop over each time step
    for j in range(dW.shape[1]):
        s = rk_step(s, dW[:, j], dt, r, sigma)

    # Final check to avoid error, non-negative values
    return np.maximum(s, 0.0)

def mlmc_sample_level_rk(S0, K, r, sigma, T, l, N_l, rng, M=2):
    """
    Generates samples for a single level 'l' of the MLMC simulation

    :param S0: Initial asset price
    :param K: Option strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity in years
    :param l: The current MLMC level where 0 is the coarsest
    :param N_l: Number of paths to simulate for this level
    :param rng: Random number generator
    :param M: Refinement factor between levels
    :return: A tuple containing:
            - Y : The array of sample values for this level.
            - cost: The computational cost per sample at this level.
    """

    n_fine = M**l
    dt_f = T / n_fine
    cost = n_fine

    # Generate a set of fine Wiener increments
    dW_f = rng.normal(0.0, sqrt(dt_f), size=(N_l, n_fine))

    # The coarsest level (l=0) is a standard Monte Carlo simulation.
    if l == 0:
        Sf = rk_step_to_maturity(S0, dW_f, dt_f, r, sigma)
        Y = exp(-r * T) * payoff_call(Sf, K)
    else:
        # For all other levels, we calculate the difference create a coarse path
        # by summing adjacent fine Wiener increments.
        dW_c = dW_f.reshape(N_l, n_fine // M, M).sum(axis=2)

        # Simulate both fine and coarse paths.
        Sf = rk_step_to_maturity(S0, dW_f, dt_f, r, sigma)
        Sc = rk_step_to_maturity(S0, dW_c, 2 * dt_f, r, sigma)

        # The sample is the difference of the discounted payoffs.
        Y = exp(-r * T) * (payoff_call(Sf, K) - payoff_call(Sc, K))

    return Y, cost

def mlmc_adaptive_rk(S0, K, r, sigma, T, eps=0.01, M=2, N0=1000, L_min=2, L_max=8, seed=42):
    """
    This implements an adaptive algorithm that automatically determines the
    number of levels and samples needed to achieve a target accuracy.

    :param S0: Initial asset price
    :param K: Option strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity in years
    :param eps: The desired standard error of the final estimate
    :param M: The refinement factor between levels
    :param N0: Initial number of samples for each level
    :param L_min: Minimum number of levels to be simulated
    :param L_max: Maximum number of levels to be simulat
    :param seed: Random number generator
    :return:The final price estimate, its standard error etc
    """


    rng = np.random.default_rng(seed)

    # Initialize arrays to store means, variances, and sample counts for each level.
    L = L_min
    means = np.zeros(L_max + 1)
    vars = np.zeros(L_max + 1)
    Nl = np.zeros(L_max + 1, dtype=int)
    costs = np.zeros(L_max + 1)

    # Initial sampling for the first few levels
    for l in range(L + 1):
        Nl[l] = max(100, N0 // (M ** l))
        Y, cost_per_sample = mlmc_sample_level_rk(S0, K, r, sigma, T, l, Nl[l], rng, M)
        means[l] = np.mean(Y)
        vars[l] = np.var(Y, ddof=1)
        costs[l] = cost_per_sample
        print(f"Level {l}: N={Nl[l]}, mean={means[l]:+.4e}, var={vars[l]:.2e}")

    # Adaptive loop
    for iteration in range(15):
        # Check if we need more levels, add finer levels if we do
        if L < L_max and abs(means[L]) > eps / sqrt(2):
            L += 1
            Nl[L] = max(100, N0 // (M ** L))
            Y, cost_per_sample = mlmc_sample_level_rk(S0, K, r, sigma, T, L, Nl[L], rng, M)
            means[L] = np.mean(Y)
            vars[L] = np.var(Y, ddof=1)
            costs[L] = cost_per_sample
            print(f"Added level {L}: N={Nl[L]}, mean={means[L]:+.4e}, var={vars[L]:.2e}")
            continue

        # Check if the desired accuracy (epsilon) has been met
        total_var = sum(vars[l] / max(Nl[l], 1) for l in range(L + 1))
        if total_var < eps**2:
            print(f"Converged: total variance {total_var:.3e} < target {eps**2:.3e}")
            break

        # Calculate optimal samples using Giles' formula,minimise cost for given variane
        N_opt = np.zeros(L + 1)
        for l in range(L + 1):
            if vars[l] > 0 and costs[l] > 0:
                N_opt[l] = max(100, int(np.sqrt(vars[l] / costs[l]) *
                                 sum(np.sqrt(vars[:L+1] * costs[:L+1])) / eps**2))

        # Run additional samples if variance is high and cost is low
        samples_added = 0
        for l in range(L + 1):
            extra = max(0, int(N_opt[l] - Nl[l]))
            # Generate new samples
            if extra > 0:
                Y_extra, _ = mlmc_sample_level_rk(S0, K, r, sigma, T, l, extra, rng, M)

                # Update mean and variance with the new samples
                if Nl[l] == 0:
                    means[l] = np.mean(Y_extra)
                    vars[l] = np.var(Y_extra, ddof=1) if extra > 1 else 0.0
                else:
                    total_samples = Nl[l] + extra
                    combined_mean = (Nl[l] * means[l] + np.sum(Y_extra)) / total_samples

                    if extra > 1:
                        var_extra = np.var(Y_extra, ddof=1)
                        vars[l] = ((Nl[l] - 1) * vars[l] + (extra - 1) * var_extra +
                                  Nl[l] * (means[l] - combined_mean)**2 +
                                  extra * (np.mean(Y_extra) - combined_mean)**2) / (total_samples - 1)
                    means[l] = combined_mean

                Nl[l] += extra
                samples_added += extra
                print(f"Level {l}: added {extra} samples, new N={Nl[l]}, mean={means[l]:+.4e}")

        if samples_added == 0:
            print("No additional samples needed - convergence achieved")
            break

    # Sum mean from all level to get final estimate
    price = sum(means[:L+1])
    se = sqrt(sum(vars[l] / max(Nl[l], 1) for l in range(L + 1)))
    total_cost = sum(Nl[l] * costs[l] for l in range(L + 1))

    print(f"\n===  rk Scheme Results ===")
    print(f"Final levels: L={L}")
    print(f"Price: {price:.6f} ± {se:.6f}")
    print(f"Total computational cost: {total_cost:.0f}")
    print(f"Samples per level: {Nl[:L+1]}")

    return price, se, L, Nl[:L+1], vars[:L+1], total_cost


def weak_order_convergence_rk(S0, K, r, sigma, T, Lmax=64, Npaths=1000000, seed=332093):
    """
    Performs a weak order convergence analysis for the rk scheme,bias vs time step size on log-log scale
    Slope is the weak convergence

    :param S0: Initial asset price
    :param K: Option strike price
    :param r: Risk-free rate
    :param sigma: Volatility
    :param T: Time to maturity in years
    :param Lmax: Maximum number of levels to be simulated
    :param Npaths: Number of paths to simulate
    :param seed: Random number generator
    :return: Result and weak order
    """

    rng = np.random.default_rng(seed)
    disc = exp(-r * T)
    exact_price = bs_price_call(S0, K, r, sigma, T)

    results = []

    # Test range of time step
    steps_list = [1, 2, 4, 8, 16, 32, 64]
    steps_list = [n for n in steps_list if n <= Lmax]

    print("===  rk Weak Order Convergence ===")

    for n in steps_list:
        dt = T / n
        # Generate fresh Brownian paths for each resolution
        dW = rng.normal(0.0, sqrt(dt), size=(Npaths, n))
        S_approx = rk_step_to_maturity(S0, dW, dt, r, sigma)
        P_approx = disc * np.maximum(S_approx - K, 0.0)

        # Calculate the bias
        mean_approx = np.mean(P_approx)
        bias = mean_approx - exact_price
        abs_bias = abs(bias)

        # Standard error of the mean
        se_mean = np.std(P_approx, ddof=1) / np.sqrt(Npaths)

        results.append((n, dt, bias, abs_bias, se_mean))
        print(f"Steps: {n:3d}, dt: {dt:.6f}, bias: {bias:+.3e}, abs_bias: {abs_bias:.3e} ± {se_mean:.3e}")

    # Fit convergence rate - use abs bias values
    dts = np.array([r[1] for r in results])
    abs_biases = np.array([r[3] for r in results])
    se_biases = np.array([r[4] for r in results])

    # Use points where bias is statistically significant
    mask = abs_biases > 2.0 * se_biases
    if np.sum(mask) >= 2:
        use_dts, use_biases = dts[mask], abs_biases[mask]
        log_dts = np.log(use_dts)

        log_biases = np.log(use_biases)
        alpha, _ = np.polyfit(log_dts, log_biases, 1)
        weak_order = alpha
        print(f"rk weak order estimate: {weak_order:.3f}")
    else:
        # This is a fall back for when the time steps (dt) are too small,else program breaks
        weak_order = 1.0
        print("Using theoretical weak order = 1.0")
    return results, weak_order

if __name__ == "__main__":
    # Parameters
    S0, K, r, sigma, T = 100.0, 100.0, 0.05, 0.20, 1.0
    exact_price = bs_price_call(S0, K, r, sigma, T)
    print(f"\nBlack–Scholes call price: {exact_price:.6f}\n")

    # Run adaptive MLMC for  RK scheme
    print("=== Adaptive MLMC for  SRK Scheme ===")
    price, se, L, Nl, var, cost = mlmc_adaptive_rk(S0, K, r, sigma, T, eps=0.01, L_max=8, N0=2000, seed=42221)

    # Weak order analysis
    print("\n")
    weak_results, weak_order = weak_order_convergence_rk(S0, K, r, sigma, T, Lmax=64, Npaths=100000, seed=21541254)

    print(f"\nFinal rk weak order estimate: {weak_order:.3f}")


Black–Scholes call price: 10.450584

=== Adaptive MLMC for  rk Scheme ===
Level 0: N=2000, mean=+9.9455e+00, var=1.93e+02
Level 1: N=1000, mean=+6.8421e-02, var=7.46e-02
Level 2: N=500, mean=+3.6417e-02, var=1.81e-02
Added level 3: N=250, mean=+2.8607e-02, var=6.60e-03
Added level 4: N=125, mean=+1.4883e-02, var=1.55e-03
Added level 5: N=100, mean=+6.6809e-03, var=2.59e-04
Level 0: added 2085385 samples, new N=2087385, mean=+1.0314e+01
Level 1: added 28019 samples, new N=29019, mean=+6.3195e-02
Level 2: added 9619 samples, new N=10119, mean=+3.5862e-02
Level 3: added 4067 samples, new N=4317, mean=+2.2422e-02
Level 4: added 1352 samples, new N=1477, mean=+1.1474e-02
Level 5: added 327 samples, new N=427, mean=+4.9404e-03
Level 0: added 134346 samples, new N=2221731, mean=+1.0315e+01
Level 1: added 2111 samples, new N=31130, mean=+6.3393e-02
Level 4: added 46 samples, new N=1523, mean=+1.1447e-02
Level 5: added 8 samples, new N=435, mean=+4.8438e-03
Converged: total variance 9.981e-05 